download requirement environment

In [1]:
pass

preprocess data

In [2]:
pass

define dataset class

In [1]:
import torch
import numpy as np
import cv2
from torch.utils.data import Dataset
import prepare_data



class RoboticsDataset(Dataset):
    def __init__(self, file_names, to_augment=False, transform=None, mode='train', problem_type=None):
        self.file_names = file_names
        self.to_augment = to_augment
        self.transform = transform
        self.mode = mode
        self.problem_type = problem_type

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_file_name = self.file_names[idx]
        image = load_image(img_file_name)
        mask = load_mask(img_file_name, self.problem_type)

        data = {"image": image, "mask": mask}
        augmented = self.transform(**data)
        image, mask = augmented["image"], augmented["mask"]

        image = image.transpose(2, 0, 1)

        if self.mode == 'train':
            if self.problem_type == 'binary':
                return image, torch.tensor(mask, dtype=torch.float32).unsqueeze(0)
            else:
                return image, torch.tensor(mask, dtype=torch.long)
        else:
            return image, str(img_file_name)


def load_image(path):
    img = cv2.imread(str(path))
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


def load_mask(path, problem_type):
    if problem_type == 'binary':
        mask_folder = 'binary_masks'
        factor = prepare_data.binary_factor
    elif problem_type == 'parts':
        mask_folder = 'parts_masks'
        factor = prepare_data.parts_factor
    elif problem_type == 'instruments':
        factor = prepare_data.instrument_factor
        mask_folder = 'instruments_masks'

    mask = cv2.imread(str(path).replace('images', mask_folder).replace('jpg', 'png'), 0)

    return (mask / factor).astype(np.uint8)

get paths of training images

In [2]:
from prepare_data import data_path
import random

def get_split(random_seeds=42):
    random.seed(random_seeds)
    train_path = data_path / 'cropped_train'
    
    train_file_names = []
    val_file_names = []

    for instrument_id in range(1, 9):
        all_file_names = list((train_path / ('instrument_dataset_' + str(instrument_id)) / 'images').glob('*'))
        random.shuffle(all_file_names)
        split_idx = int(len(all_file_names) * 0.8)
        train_file_names.extend(all_file_names[:split_idx])
        val_file_names.extend(all_file_names[split_idx:])

    return train_file_names, val_file_names

get dataloader

In [10]:
train_crop_height = 1024
train_crop_width = 1280
val_crop_height = 1024
val_crop_width = 1280
workers = 12
batch_size = 8
problem_type = 'binary'

In [11]:
from torch.utils.data import DataLoader
def make_loader(file_names, shuffle=False, transform=None, problem_type=problem_type, batch_size=1):
        return DataLoader(
            dataset=RoboticsDataset(file_names, transform=transform, problem_type=problem_type),
            shuffle=shuffle,
            num_workers=workers,
            batch_size=batch_size,
            pin_memory=torch.cuda.is_available()
        )

In [12]:
from albumentations import (
    HorizontalFlip,
    VerticalFlip,
    Normalize,
    Compose,
    PadIfNeeded,
    RandomCrop,
    CenterCrop
)
def train_transform(p=1):
    return Compose([
        PadIfNeeded(min_height=train_crop_height, min_width=train_crop_width, p=1),
        RandomCrop(height=train_crop_height, width=train_crop_width, p=1),
        VerticalFlip(p=0.5),
        HorizontalFlip(p=0.5),
        Normalize(p=1)
    ], p=p)

def val_transform(p=1):
    return Compose([
        PadIfNeeded(min_height=val_crop_height, min_width=val_crop_width, p=1),
        CenterCrop(height=val_crop_height, width=val_crop_width, p=1),
        Normalize(p=1)
    ], p=p)

In [13]:
train_file_names, val_file_names = get_split()
train_loader = make_loader(train_file_names, shuffle=True, transform=train_transform(p=1), problem_type=problem_type,batch_size=batch_size)
valid_loader = make_loader(val_file_names, transform=val_transform(p=1), problem_type=problem_type,batch_size=batch_size)

get model

In [14]:
num_classes = 1
device_ids = [0]

In [15]:
from models import UNet11, LinkNet34, UNet, UNet16, AlbuNet
from torch import nn
from unetplusplus import UnetPlusPlus
from model1 import LinkNet34_modified

model_name = 'LinkNet34_modified'
model = LinkNet34_modified(num_classes=num_classes)
if torch.cuda.is_available():
    model = nn.DataParallel(model, device_ids=device_ids).cuda()

loss function

In [16]:
from loss import LossBinary, LossMulti
jaccard_weight = 0.3
loss = LossBinary(jaccard_weight=jaccard_weight)

benchmark

In [17]:
import torch.backends.cudnn as cudnn
import torch.backends.cudnn

cudnn.benchmark = True

checkpoints folder

In [18]:
import json

In [19]:
root = 'runs/debug'
# root.joinpath('params.json').write_text(json.dumps(vars(args), indent=True, sort_keys=True))

train function

In [20]:
from datetime import datetime
from pathlib import Path

import random
import numpy as np

import torch
import tqdm

In [21]:
def cuda(x):
    return x.to('cuda', non_blocking=True) if torch.cuda.is_available() else x
def write_event(log, step, **data):
    data['step'] = step
    data['dt'] = datetime.now().isoformat()
    log.write(json.dumps(data, sort_keys=True))
    log.write('\n')
    log.flush()

In [22]:
def train(model, criterion, train_loader, valid_loader, validation, init_optimizer, n_epochs=1, lr=0.0001, batch_size=1, fold=None,
          num_classes=None, root='runs/debug',model_name=model_name):
    lr = lr
    n_epochs = n_epochs
    optimizer = init_optimizer(lr)

    root = Path(root)
    model_path = root / f'{model_name}.pt'
    if model_path.exists():
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
    else:
        epoch = 1
        step = 0

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
    }, str(model_path))

    report_each = 10
    log = root.joinpath('train.log').open('at', encoding='utf8')
    valid_losses = []
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        random.seed()
        tq = tqdm.tqdm(total=(len(train_loader) * batch_size))
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses = []
        tl = train_loader
        mean_loss = 0
        for i, (inputs, targets) in enumerate(tl):
            inputs = cuda(inputs)

            with torch.no_grad():
                targets = cuda(targets)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            batch_size = inputs.size(0)
            loss.backward()
            optimizer.step()
            step += 1
            tq.update(batch_size)
            losses.append(loss.item())
            mean_loss = np.mean(losses[-report_each:])
            tq.set_postfix(loss='{:.5f}'.format(mean_loss))
            if i and i % report_each == 0:
                write_event(log, step, loss=mean_loss)
        write_event(log, step, loss=mean_loss)
        tq.close()
        save(epoch + 1)
        valid_metrics = validation(model, criterion, valid_loader, num_classes)
        write_event(log, step, **valid_metrics)
        valid_loss = valid_metrics['valid_loss']
        valid_losses.append(valid_loss)


validation function

In [23]:
from validation import validation_binary, validation_multi
valid = validation_binary

optimizer

In [24]:
from torch.optim import Adam
optimizer = lambda lr: Adam(model.parameters(), lr=lr)

train

In [25]:
train(
    model=model,
    criterion=loss,
    train_loader=train_loader,
    valid_loader=valid_loader,
    validation=valid,
    init_optimizer=optimizer,
    n_epochs=15,
    lr=0.0001,
    batch_size=batch_size,
    num_classes=num_classes,
    model_name = model_name
)

Epoch 1, lr 0.0001:   0%|          | 0/1440 [00:00<?, ?it/s]/root/autodl-tmp/mldl/code/model1.py:92: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  out = F.upsample(out, size=(h, w), mode="bilinear")
Epoch 1, lr 0.0001: 100%|██████████| 1440/1440 [00:57<00:00, 25.10it/s, loss=0.32689]


Valid loss: 0.34055, jaccard: 0.01944


Epoch 2, lr 0.0001: 100%|██████████| 1440/1440 [00:50<00:00, 28.63it/s, loss=0.07962]


Valid loss: 0.08544, jaccard: 0.88614


Epoch 3, lr 0.0001: 100%|██████████| 1440/1440 [00:50<00:00, 28.31it/s, loss=0.06414]


...

Valid loss: 0.05810, jaccard: 0.91594


Epoch 14, lr 0.0001: 100%|██████████| 1440/1440 [00:51<00:00, 28.19it/s, loss=0.03520]


Valid loss: 0.05764, jaccard: 0.91368


Epoch 15, lr 0.0001: 100%|██████████| 1440/1440 [00:51<00:00, 28.16it/s, loss=0.03723]


Valid loss: 0.05123, jaccard: 0.92908


generate mask using the model

In [42]:
output_path = f'predictions/{model_name}/binary'
workers = 12
from prepare_data import (original_height,
                          original_width,
                          h_start, w_start
                          )

In [43]:
def img_transform(p=1):
    return Compose([
        Normalize(p=1)
    ], p=p)

In [44]:
from torch.nn import functional as F
def predict(model, from_file_names, batch_size, to_path, problem_type, img_transform,workers=workers,):
    loader = DataLoader(
        dataset=RoboticsDataset(from_file_names, transform=img_transform, mode='predict', problem_type=problem_type),
        shuffle=False,
        batch_size=batch_size,
        num_workers=workers,
        pin_memory=torch.cuda.is_available()
    )

    with torch.no_grad():
        for batch_num, (inputs, paths) in enumerate(tqdm.tqdm(loader, desc='Predict')):
            inputs = cuda(inputs)

            outputs = model(inputs)

            for i, image_name in enumerate(paths):
                if problem_type == 'binary':
                    factor = prepare_data.binary_factor
                    t_mask = (F.sigmoid(outputs[i, 0]).data.cpu().numpy() * factor).astype(np.uint8)
                elif problem_type == 'parts':
                    factor = prepare_data.parts_factor
                    t_mask = (outputs[i].data.cpu().numpy().argmax(axis=0) * factor).astype(np.uint8)
                elif problem_type == 'instruments':
                    factor = prepare_data.instrument_factor
                    t_mask = (outputs[i].data.cpu().numpy().argmax(axis=0) * factor).astype(np.uint8)

                h, w = t_mask.shape

                full_mask = np.zeros((original_height, original_width))
                full_mask[h_start:h_start + h, w_start:w_start + w] = t_mask

                instrument_folder = Path(paths[i]).parent.parent.name

                (to_path / instrument_folder).mkdir(exist_ok=True, parents=True)

                cv2.imwrite(str(to_path / instrument_folder / (Path(paths[i]).stem + '.png')), full_mask)

In [45]:
val_file_names
print('num file_names = {}'.format(len(val_file_names)))
output_path = Path(output_path)
output_path.mkdir(exist_ok=True, parents=True)

predict(model, val_file_names, batch_size, output_path, problem_type=problem_type,img_transform=img_transform(p=1))

num file_names = 360


Predict: 100%|██████████| 90/90 [00:19<00:00,  4.52it/s]


evaluate

In [46]:
def jaccard(y_true, y_pred):
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    return (intersection + 1e-15) / (union + 1e-15)


def dice(y_true, y_pred):
    return (2 * (y_true * y_pred).sum() + 1e-15) / (y_true.sum() + y_pred.sum() + 1e-15)

In [47]:
import os
from prepare_data import height, width, h_start, w_start

target_path = f'predictions/{model_name}'
train_path = 'data/cropped_train'

In [48]:
result_dice = []
result_jaccard = []
if problem_type == 'binary':
    for instrument_id in tqdm.tqdm(range(1, 9)):
        instrument_dataset_name = 'instrument_dataset_' + str(instrument_id)

        pred_folder_name = (Path(target_path) / 'binary' / instrument_dataset_name)
        if not os.path.exists(pred_folder_name):
            continue

        for file_name in (Path(train_path) / instrument_dataset_name / 'binary_masks').glob('*'):
            pred_file_name = (Path(target_path) / 'binary' / instrument_dataset_name / file_name.name)
            if not os.path.exists(pred_file_name):
                continue
            
            y_true = (cv2.imread(str(file_name), 0) > 0).astype(np.uint8)

            pred_image = (cv2.imread(str(pred_file_name), 0) > 255 * 0.5).astype(np.uint8)
            y_pred = pred_image[h_start:h_start + height, w_start:w_start + width]

            result_dice += [dice(y_true, y_pred)]
            result_jaccard += [jaccard(y_true, y_pred)]

100%|██████████| 8/8 [00:06<00:00,  1.15it/s]


In [28]:
print('Dice = ', np.mean(result_dice), np.std(result_dice))
print('Jaccard = ', np.mean(result_jaccard), np.std(result_jaccard))

Dice =  0.9306508661052798 0.08574437766033535
Jaccard =  0.8792055832122158 0.11364465961032456


In [49]:
model = LinkNet34(1)
model_name = 'LinkNet34'
model_path = 'data/models/LinkNet34/LinkNet34.pt'
problem_type = 'binary'

state = torch.load(str(model_path))
state = {key.replace('module.', ''): value for key, value in state['model'].items()}
model.load_state_dict(state)

if torch.cuda.is_available():
    model.cuda()

/tmp/ipykernel_2458/3808355107.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(str(model_path))


In [30]:
target_path = 

SyntaxError: invalid syntax (283474602.py, line 1)